![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+GenAI%2FEvaluation&file=Optimize+Prompts+Using+Evaluation+Metrics.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Evaluation/Optimize%20Prompts%20Using%20Evaluation%20Metrics.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520GenAI%2FEvaluation%2FOptimize%2520Prompts%2520Using%2520Evaluation%2520Metrics.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Evaluation/Optimize%20Prompts%20Using%20Evaluation%20Metrics.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20GenAI/Evaluation/Optimize%20Prompts%20Using%20Evaluation%20Metrics.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Optimize Prompts Using Evaluation Metrics

TBD


[Optimize Prompts](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-optimizer)


[Evaluation For GenAI](./Evaluation%20For%20GenAI.ipynb)

---
## Colab Setup

To run this notebook in Colab run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

Not a Colab Environment


---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform', '1.78.0'),
    ('google.cloud.storage', 'google-cloud-storage'),
    ('pandas', 'pandas')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [4]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

inputs:

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [7]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'prompt-optimization'

BUCKET = PROJECT_ID # change to Bucket name if not the same as the Project ID

packages:

In [32]:
# Python standard library imports:
import json, io, requests, sys, types

# package imports
from IPython.display import Markdown
import pandas as pd

# vertex ai imports
from google.cloud import aiplatform
from google.cloud import storage
import vertexai


#import vertexai.generative_models # for Gemini Models
#import vertexai.evaluation 

In [15]:
aiplatform.__version__

'1.78.0'

clients:

In [16]:
vertexai.init(project = PROJECT_ID, location = REGION)
gcs = storage.Client(project = PROJECT_ID)

---
## Optimize Prompts

### Load The Code

In [27]:
module_name = 'vapo_lib'
url = 'https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/prompt_optimizer/vapo_lib.py'
response = requests.get(url)
vapo_lib = types.ModuleType(module_name)
vapo_lib.__file__ = f'<remote>/{module_name}.py'
sys.modules[module_name] = module
exec(response.text, vapo_lib.__dict__)

https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-optimizer#how_optimization_works
https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-optimizer#sdk_1

In [28]:
SOURCE_MODEL = "gemini-1.5-flash-001" # or provide ground truth
TARGET_MODEL = "gemini-1.5-flash-002" 
OPTIMIZATION_MODE = "instruction"
EVAL_METRICS = "coherence"]

In [29]:
SYSTEM_INSTRUCTIONS = "Write poems in the style requested based on the topic provided"

In [30]:
PROMPT_TEMPLATE = "Write a {type} about {topic}."

In [34]:
prompt_parameters = [
    dict(type = 'Haiku', topic = 'Lego'),
    dict(type = 'Sonnet', topic = 'Lego'),
    dict(type = 'Limerick', topic = 'Lego'),
    dict(type = 'Acrostic', topic = 'Lego'),
    dict(type = 'Ode', topic = 'Lego')
]

In [36]:
bucket = gcs.bucket(BUCKET)
blob = bucket.blob(f'{SERIES}/{EXPERIMENT}/prompt_parameters.jsonl')
with io.StringIO() as jsonl_file:
    for item in prompt_parameters:
        json.dump(item, jsonl_file)
        jsonl_file.write('\n')
    blob.upload_from_string(jsonl_file.getvalue(), content_type = 'application/jsonl')

In [45]:
vapo_lib.is_run_target_required(
    eval_metric_types = EVAL_METRICS,
    source_model = SOURCE_MODEL
)

False

In [46]:
vapo_lib.validate_prompt_and_data(
    template = '\n'.join([SYSTEM_INSTRUCTIONS, PROMPT_TEMPLATE]),
    dataset_path = f'gs://{bucket.name}/{blob.name}',
    placeholder_to_content = '{}',
    label_enforced = vapo_lib.is_run_target_required(
        eval_metric_types = EVAL_METRICS,
        source_model = SOURCE_MODEL
    )
)

In [52]:
job = vapo_lib.run_apd(
    config = dict(
        project = PROJECT_ID,
        system_instruction = SYSTEM_INSTRUCTIONS,
        prompt_template = PROMPT_TEMPLATE,
        target_model = TARGET_MODEL,
        target_model_location = REGION, 
        eval_metrics_types = ['coherence', 'fluency'],
        eval_metrics_weights = [.5, .5],
        aggregation_type = 'weighted_sum',
        source_model = SOURCE_MODEL,
        optimization_mode = OPTIMIZATION_MODE,
        input_data_path = f'gs://{bucket.name}/{blob.name}',
        output_path = f'gs://{bucket.name}/{SERIES}/{EXPERIMENT}'
    ),
    bucket_uri = f'gs://{bucket.name}/{SERIES}/{EXPERIMENT}',
    display_name = 'lego_lyrics'
)



Job display name: lego_lyrics
Creating CustomJob
CustomJob created. Resource name: projects/1026793852137/locations/us-central1/customJobs/2919439182629699584
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/1026793852137/locations/us-central1/customJobs/2919439182629699584')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/2919439182629699584?project=1026793852137


In [53]:
job.name

'2919439182629699584'